# Use of logs for getting all the metrics at the end of Epoch

In [1]:
from twilio.rest import Client

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.datasets import make_classification

import tensorflow as tf
from tensorflow import keras 

D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [2]:
def send_message(text):
    account_sid = 'AC627ada27fffdce01d175164bcba0d23c' #Can be obtained from Twilio Console
    auth_token = '729c156bf50bc7bc1fb49c2256cd9836'  #Can be obtained from Twilio Console
    client = Client(account_sid, auth_token)

    message = client.messages \
    .create(
         from_='whatsapp:+14155238886',
         body=text,
         to='whatsapp:+917400452552'
     )

In [3]:
class LossAndErrorPrintingCallback(keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        send_message("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        send_message("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        send_message(
            "The average loss for epoch {} is {:7.2f} "
            "and mean absolute error is {:7.2f}.".format(
                epoch, logs["loss"], logs["mean_absolute_error"]
            )
        )

In [4]:
cb = LossAndErrorPrintingCallback() #Creating the Callback

In [5]:
x,y = make_classification(n_samples=1000, n_classes=2, n_features=20)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [6]:
def create_model():
    model = Sequential()
    model.add(Dense(60, input_dim=20, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss="mean_squared_error", optimizer=keras.optimizers.RMSprop(learning_rate=0.1), metrics=["mean_absolute_error"])

    return model

model = create_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
model.fit(x_train,y_train,batch_size=32, epochs=20, callbacks=[cb], verbose=1) #Fitting the model

Epoch 1/20
700/700 [==============================] - 28s 40ms/sample - loss: 0.1498 - mean_absolute_error: 0.1768
Epoch 2/20
480/700 [===================>..........] - ETA: 7s - loss: 0.1335 - mean_absolute_error: 0.1385

KeyboardInterrupt: 

In [8]:
res = model.evaluate(x_test, y_test, batch_size=32, verbose=1, callbacks=[cb])
res = model.predict(x_test, batch_size=32, callbacks=[cb])

ConnectionError: HTTPSConnectionPool(host='api.twilio.com', port=443): Max retries exceeded with url: /2010-04-01/Accounts/AC627ada27fffdce01d175164bcba0d23c/Messages.json (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000026402BBC448>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))